<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Style-Transfer" data-toc-modified-id="Style-Transfer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Style Transfer</a></span><ul class="toc-item"><li><span><a href="#More-Example-Results" data-toc-modified-id="More-Example-Results-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>More Example Results</a></span><ul class="toc-item"><li><span><a href="#Same-content-image" data-toc-modified-id="Same-content-image-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Same content image</a></span></li><li><span><a href="#Same-style-image" data-toc-modified-id="Same-style-image-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Same style image</a></span></li></ul></li><li><span><a href="#Example-Code" data-toc-modified-id="Example-Code-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Example Code</a></span><ul class="toc-item"><li><span><a href="#Defined-References" data-toc-modified-id="Defined-References-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Defined References</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Image-Process" data-toc-modified-id="Image-Process-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Image Process</a></span></li><li><span><a href="#Model---Style-Transfer" data-toc-modified-id="Model---Style-Transfer-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Model - Style Transfer</a></span></li></ul></li></ul></li><li><span><a href="#Other-Technique:-CycleGAN" data-toc-modified-id="Other-Technique:-CycleGAN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Other Technique: CycleGAN</a></span></li></ul></div>

# Style Transfer

[A Neural Algorithm of Artistic Style](http://arxiv.org/abs/1508.06576)

Keras Implementation: https://keras.io/examples/neural_style_transfer/

PyTorch Tutorial (where some of the images in example code came from): https://github.com/udacity/deep-learning-v2-pytorch/tree/master/style-transfer

<img src='images/starry-mona.jpg'/>

## More Example Results

### Same content image

In [1]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr"><a href="https://twitter.com/hashtag/100DaysOfCode?src=hash&amp;ref_src=twsrc%5Etfw">#100DaysOfCode</a> <a href="https://twitter.com/hashtag/R2D06?src=hash&amp;ref_src=twsrc%5Etfw">#R2D06</a><br>Learned image style transfer with CNNs using PyTorch (from Udacity&#39;s free course <a href="https://t.co/E54OgWJWIg">https://t.co/E54OgWJWIg</a>)<br>I&#39;m definitely having way too much fun messing w/images! Will to have to go through the concepts again since I rushed it to play around <a href="https://t.co/uM5OWMRzeM">pic.twitter.com/uM5OWMRzeM</a></p>&mdash; Victor Geislinger (@MrGeislinger) <a href="https://twitter.com/MrGeislinger/status/1124169701775380480?ref_src=twsrc%5Etfw">May 3, 2019</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

### Same style image

<img src='images/zebra-final-all.png'/>

> GIF version: https://twitter.com/MrGeislinger/status/1141891997235466240

<img src='images/sky-final-all.png'/>

## Example Code

In [ ]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time

from keras.applications import vgg19
from keras import backend as K

In [ ]:
def load_image(img_path, max_size=400, shape=None):
    ''' Load in and transform an image, making sure the image
       is <= 400 pixels in the x-y dims.'''
    if "http" in img_path:
        response = requests.get(img_path)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(img_path).convert('RGB')
    
    # large images will slow down processing
    if max(image.size) > max_size:
        size = max_size
    else:
        size = max(image.size)
    
    if shape is not None:
        size = shape
        
    in_transform = transforms.Compose([
                        transforms.Resize(size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = in_transform(image)[:3,:,:].unsqueeze(0)
    
    return image

### Defined References

In [ ]:
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

### Parameters

In [ ]:
base_image_path = 'images-style-transfer/mona_lisa.jpg'
style_reference_image_path = 'images-style-transfer/starry_night.jpg'

In [ ]:
result_prefix = 'result_'
iterations = 25

In [ ]:
total_variation_weight = 1.0
content_weight = 1.0
style_weight = 1.0e6

In [ ]:
# dimensions of the generated picture.
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [ ]:
style_weights = {
    'block1_conv1': 1.0,
    'block2_conv1': 0.8,
    'block3_conv1': 0.2,
    'block4_conv1': 0.2,
    'block5_conv1': 0.2 
}

### Image Process

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image


def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

# this will contain our generated image
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

### Model - Style Transfer

In [ ]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [ ]:
# compute the neural style loss
# first we need to define 4 util functions

# the gram matrix of an image tensor (feature-wise outer product)


def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    
    gram = K.dot(features, K.transpose(features))  

    return gram

# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image


def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols

    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image


def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent


def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

# combine these loss functions into a single scalar
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)


for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_weights[layer_name] * style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)


def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    if (i % 5) == 0:
        img = deprocess_image(x.copy())
        fname = f'{base_image_path}-to-{style_reference_image_path}-{i:03}.png'
        save_img(fname, img)
        end_time = time.time()
        print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

# Other Technique: CycleGAN

Original paper: https://arxiv.org/abs/1703.10593

![](images/horse2zebra_1.png)
![](images/horse2zebra_2.png)
> TensorFlow tutorial: https://www.tensorflow.org/tutorials/generative/cyclegan